In [1]:
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
from pprint import pprint

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
import nltk
#nltk.download('stopwords')
#nltk.download()

In [6]:
from nltk.corpus import stopwords


In [7]:
import spacy

In [8]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [10]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [11]:
stopwords.words('spanish')

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [12]:
df = pd.read_json('../data/raw/newsgroups.json')
print(df.target_names.unique())
print(len(df))

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']
11314


In [13]:
df = df[:1000]

In [14]:
#len(df)
#df.describe()
df

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
995,From: ibeshir@nyx.cs.du.edu (Ibrahim)\nSubject...,6,misc.forsale
996,From: anwar+@cs.cmu.edu (Anwar Mohammed)\nSubj...,17,talk.politics.mideast
997,From: DJCOHEN@YaleVM.YCC.Yale.Edu (Daniel Cohe...,4,comp.sys.mac.hardware
998,From: geb@cs.pitt.edu (Gordon Banks)\nSubject:...,13,sci.med


In [15]:
df.loc[0,'content']

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [16]:
# Convertir a una lista
data = df.content.values.tolist()
pprint(data[:1])

["From: lerxst@wam.umd.edu (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


In [17]:
# Eliminar emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
pprint(data[:1])

["From: (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


In [18]:
# Eliminar newlines
data = [re.sub(r'\s+', ' ', sent) for sent in data]
pprint(data[:1])

["From: (where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: "
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [19]:
# Eliminar comillas
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [20]:
def sent_to_words(sentences):
    for sentence in sentences:
    # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [21]:
# Construimos modelos de bigrams y trigrams
# https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10)
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)  

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [22]:
print(bigram[data_words[0]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting', 'host', 'rac', 'wam', 'umd', 'edu_organization', 'university_of', 'maryland', 'college', 'park', 'lines', 'was_wondering', 'if_anyone', 'out_there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other_day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in_addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the_rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if_anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or_whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please_mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [23]:
print(trigram[bigram[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac', 'wam', 'umd', 'edu_organization_university_of', 'maryland', 'college', 'park', 'lines', 'was_wondering_if_anyone', 'out_there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other_day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to_be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in_addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the_rest_of', 'the', 'body', 'this', 'is', 'all', 'know', 'if_anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or_whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please_mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [24]:
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting', 'host', 'rac', 'wam', 'umd', 'edu_organization', 'university_of', 'maryland', 'college', 'park', 'lines', 'was_wondering', 'if_anyone', 'out_there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other_day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in_addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the_rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if_anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or_whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please_mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']
['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', '

In [25]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [27]:
data_words_nostops[0]

['wheres',
 'thing',
 'car',
 'nntp',
 'posting',
 'host',
 'rac',
 'wam',
 'umd',
 'organization',
 'university',
 'maryland',
 'college',
 'park',
 'lines',
 'wondering',
 'anyone',
 'could',
 'enlighten',
 'car',
 'saw',
 'day',
 'door',
 'sports',
 'car',
 'looked',
 'late',
 'early',
 'called',
 'bricklin',
 'doors',
 'really',
 'small',
 'addition',
 'front',
 'bumper',
 'separate',
 'rest',
 'body',
 'know',
 'anyone',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'years',
 'production',
 'car',
 'made',
 'history',
 'whatever',
 'info',
 'funky',
 'looking',
 'car',
 'please',
 'mail',
 'thanks',
 'il',
 'brought',
 'neighborhood',
 'lerxst']

In [28]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [29]:
data_words_bigrams[0]

['wheres',
 'thing',
 'car',
 'nntp_posting',
 'host',
 'rac',
 'wam',
 'umd',
 'organization_university',
 'maryland',
 'college',
 'park',
 'lines',
 'wondering',
 'anyone',
 'could',
 'enlighten',
 'car',
 'saw',
 'day',
 'door',
 'sports_car',
 'looked',
 'late',
 'early',
 'called',
 'bricklin',
 'doors',
 'really',
 'small',
 'addition',
 'front',
 'bumper',
 'separate',
 'rest',
 'body',
 'know',
 'anyone',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'years',
 'production',
 'car',
 'made',
 'history',
 'whatever',
 'info',
 'funky',
 'looking',
 'car',
 'please_mail',
 'thanks',
 'il',
 'brought',
 'neighborhood',
 'lerxst']

In [29]:
# python3 -m spacy download en_core_web_lg
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

ValueError: Expected object or value

### <span style="color:blue">HASTA AQUI PREPROCESAMIENTO</span> 


In [31]:
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)

In [32]:
for key, value in id2word.items():
    print(key, value)

0 addition
1 body
2 bring
3 bumper
4 call
5 car
6 could
7 door
8 early
9 engine
10 enlighten
11 front
12 funky
13 history
14 host
15 info
16 know
17 late
18 line
19 look
20 make
21 model
22 name
23 neighborhood
24 nntp_poste
25 production
26 really
27 rest
28 see
29 separate
30 small
31 spec
32 thank
33 where
34 wonder
35 year
36 acceleration
37 adapter
38 add
39 answer
40 article
41 attain
42 base
43 brave
44 brief
45 card
46 clock
47 detail
48 do
49 especially
50 experience
51 fair
52 final
53 floppy
54 floppy_disk
55 functionality
56 guy_kuo
57 heat
58 hour
59 keyword
60 knowledge
61 message
62 network
63 next
64 number
65 oscillator
66 per_day
67 please_send
68 poll
69 procedure
70 rate
71 report
72 request
73 share
74 si
75 sink
76 soul
77 speed
78 summarize
79 summary
80 top
81 two_day
82 upgrade
83 usage
84 access
85 active
86 actually
87 advance
88 anymore
89 appearence
90 back
91 be
92 bite
93 breifly
94 bunch
95 computer
96 computer_network
97 conviction
98 daily
99 dangerous

1145 dos_window
1146 excuse
1147 fairly
1148 font
1149 hpoe
1150 monitor
1151 ms_window
1152 normal
1153 simplistic
1154 spacify
1155 trivial
1156 upper
1157 world
1158 blind
1159 cartridge
1160 computer_science
1161 definitely
1162 involve
1163 laboratory
1164 none
1165 scsi_problem
1166 silverline
1167 solution
1168 specific
1169 venerable
1170 accessible
1171 adresse
1172 applicable
1173 avoid
1174 care
1175 hassle
1176 impressive
1177 input
1178 interest
1179 post_summary
1180 purchasing
1181 sufficient
1182 unix
1183 vendor
1184 voice
1185 air
1186 ambient
1187 appropriate
1188 away
1189 boil
1190 boiler
1191 boiling
1192 canadian
1193 closed
1194 cold
1195 concrete
1196 condenser
1197 contamination
1198 cool
1199 cooling_tower
1200 crud
1201 cycle
1202 danger
1203 date_fri
1204 dump
1205 ever
1206 excerpts
1207 explaination
1208 fire
1209 gripe
1210 hence
1211 hot
1212 icing
1213 industry
1214 leave
1215 line_communication
1216 middle
1217 nuclear_plant
1218 nuclear_power
1219 nu

2331 heart
2332 automobile
2333 depletion
2334 greatly_appreciate
2335 guilty
2336 partially
2337 ancient
2338 apply
2339 atom
2340 cannot_imagine
2341 centre
2342 comprehend
2343 comprise
2344 concept
2345 continuous
2346 continuum
2347 deletion
2348 endless
2349 evolve
2350 imagine
2351 infinietely
2352 infinitely
2353 infinity
2354 knowledgeable
2355 mathematical
2356 precicely
2357 programmer
2358 sunlight
2359 com
2360 distribution_world
2361 domain
2362 infamous
2363 pave
2364 alcohol
2365 blue
2366 boy
2367 compound
2368 confirm
2369 cult
2370 dealing
2371 evasion
2372 evidence
2373 involde
2374 methamphetamine
2375 protective
2376 scope
2377 tobacco
2378 atheism
2379 atheist
2380 belief
2381 bloody
2382 bottom
2383 butter
2384 caste
2385 choice
2386 colonization
2387 contentsthan
2388 cow
2389 crusade
2390 definately
2391 destruction
2392 dish
2393 european
2394 false
2395 fanatic
2396 flame
2397 genocide
2398 hatred
2399 holy
2400 horror
2401 hunt
2402 ideological
2403 ideolog

3105 refresh
3106 segmented
3107 somehow
3108 worthwhile
3109 annoy
3110 apparently
3111 bio
3112 button
3113 consistently
3114 crashing_please
3115 diagnostic
3116 exe
3117 fine
3118 flaky
3119 gpf
3120 gpfs
3121 ide
3122 ie
3123 manage
3124 oracle_corporation
3125 package
3126 parity
3127 pro
3128 reboot
3129 reproducible
3130 turning
3131 unauthenticated
3132 windows_keep
3133 fairing
3134 guage
3135 magnife
3136 melt
3137 motorcycle
3138 plexi
3139 boulder
3140 colorado_bould
3141 golden
3142 retriever
3143 sidecar
3144 tricky
3145 alert
3146 approval
3147 approve
3148 commission
3149 community
3150 constituency
3151 cumulative
3152 demographic
3153 desk
3154 executive
3155 foreclose
3156 impact
3157 intentionally
3158 naval
3159 nomination
3160 particularly
3161 percentage
3162 personnel
3163 plausible
3164 realignment
3165 reserve
3166 reservist
3167 role
3168 significant
3169 suggest
3170 thesis
3171 warrant
3172 weight
3173 actualy
3174 callback
3175 coding
3176 compensation
31

3715 computer_inc
3716 decline
3717 entertainer
3718 exception
3719 gondolin
3720 million_dollar
3721 pirate
3722 psbdn
3723 remeber
3724 take_away
3725 studio
3726 alias
3727 computer_service
3728 couldnt_find
3729 wanted
3730 behaviour
3731 condem
3732 disproof
3733 familiar
3734 long_time
3735 pertain
3736 rephrase
3737 beer
3738 bread
3739 breakfast
3740 calcium
3741 cereal
3742 chelate
3743 chelated
3744 chemistry
3745 chromium
3746 compatibility
3747 diagnose
3748 discuss
3749 excess
3750 food
3751 gate
3752 gateway
3753 grain
3754 honey
3755 hypoglycemia
3756 hypoglycemic
3757 manganese
3758 pancreatic
3759 potassium
3760 practitioner
3761 ron_roth
3762 rosemail
3763 specialty
3764 sugar
3765 supplement
3766 zinc
3767 bat
3768 bb
3769 finish
3770 index
3771 offensive
3772 pit
3773 pitcher
3774 pose
3775 diamond
3776 drv
3777 faint
3778 latest_driver
3779 unaware
3780 winword
3781 wspdpsf
3782 apartment
3783 carport
3784 hurricane
3785 practically
3786 relative
3787 stash
3788 sy

...

14944 eico
14945 schematic
14946 furthur
14947 petcock
14948 tping
14949 supervga
14950 appreasiate
14951 directional
14952 fitanide
14953 alway
14954 architectural
14955 transidental
14956 bimmer
14957 astronomically
14958 beleif
14959 elapse
14960 incedently
14961 rabbitball
14962 imake
14963 aneutronic
14964 antimatter
14965 antiproton
14966 appendice
14967 appendicie
14968 avation
14969 bibliographic
14970 booklet
14971 cartography
14972 cesium
14973 chock
14974 collectible
14975 compactness
14976 compressible
14977 conceptual
14978 concurrency
14979 daydreamer
14980 deformation
14981 descriptor
14982 dieter
14983 electrostatic
14984 farside
14985 fennelly
14986 fictitious
14987 fudge
14988 geodesy
14989 geostationary
14990 gigawatt
14991 guardians
14992 interplanetary
14993 intersteller
14994 kluwer
14995 lambeck
14996 leech
14997 lightne
14998 lightsail
14999 megawat
15000 metastable
15001 mf
15002 miniature
15003 modeller
15004 modest
15005 monogram
15006 nearside
15007 nontechn

16103 receipt
16104 unchanged
16105 bragging
16106 hysterically
16107 played
16108 righty
16109 deactivate
16110 finnally
16111 nichael
16112 aphelion
16113 eccentricity
16114 jupiter
16115 perijove
16116 specialix
16117 streetcar
16118 excitotoxicity
16119 neuron
16120 altare
16121 beahvioral
16122 burglary
16123 fatality
16124 fivefold
16125 handgun_control
16126 knife_attack
16127 minimally
16128 preferred
16129 robbery
16130 subdivide
16131 turf
16132 violent_crime
16133 arbiter
16134 behaviorist
16135 cognitivist
16136 desideratum
16137 fashionable
16138 grounding
16139 impeccable
16140 insistence
16141 stultification
16142 unionism
16143 msdos_upload
16144 rend
16145 renderer
16146 reupload
16147 cruddy
16148 jj
16149 wobble
16150 elbow
16151 esecially
16152 figuring
16153 kms
16154 sqkm
16155 accuser
16156 doctoring
16157 graduate_student
16158 hereafter
16159 immmense
16160 incompetent
16161 pretext
16162 roehm
16163 shred
16164 shrewd
16165 taint
16166 thereafter
16167 blockad

16725 inexperienced
16726 infancy
16727 infeasible
16728 insidious
16729 inspect
16730 intermediate
16731 intertwined
16732 inundate
16733 invulnerability
16734 jokingly
16735 lax
16736 logging
16737 logout
16738 malevolent
16739 memorable
16740 millenium
16741 munge
16742 nameserver
16743 nntp
16744 nonexistent
16745 nonlocal
16746 nonuniformity
16747 objectionable
16748 obscurely
16749 origination
16750 parsing
16751 passwd
16752 pathological
16753 pervasive
16754 plonk
16755 pornographic
16756 postmaster
16757 prepende
16758 prohibitive
16759 provoking
16760 pseudonym
16761 pseudonymity
16762 punitive
16763 reallocate
16764 refined
16765 remaile
16766 remailer
16767 responsibiltie
16768 reviewer
16769 rhost
16770 routing
16771 rudely
16772 signficant
16773 slang
16774 spool
16775 steganography
16776 stigma
16777 substring
16778 suffix
16779 superhighway
16780 surreptitious
16781 system_administrator
16782 toward
16783 troubleshoot
16784 troubleshooting
16785 umask
16786 un_important

17353 substantive
17354 mail_address
17355 blurb
17356 grate
17357 morganti
17358 blend
17359 emplacement
17360 greater
17361 morass
17362 palatable
17363 regrettable
17364 resistrance
17365 shai_guday
17366 tyoe
17367 anti_israel
17368 blah
17369 blathering
17370 catfight
17371 cork
17372 damning
17373 embitter
17374 handwrite
17375 indictment
17376 leftist
17377 pail
17378 selecting
17379 spicy
17380 chinese_food
17381 ununsual
17382 and
17383 microcontroller
17384 cc_buffalo
17385 ubvmsd
17386 acknowledgment
17387 allowable
17388 display_type
17389 djgpp
17390 fac
17391 formt
17392 grn
17393 iff
17394 img
17395 ish
17396 jpeg
17397 macbinary
17398 sort_method
17399 truecolor
17400 writng
17401 xc
17402 xx
17403 oomph
17404 platter
17405 lciii
17406 broadening
17407 encroachment
17408 inequity
17409 intrusion
17410 lifeblood
17411 monograph
17412 thrice
17413 blue_max
17414 red_max
17415 rgb_gray_map
17416 standard_colormaps
17417 xffffffff
17418 xstdcmap
17419 flaunt
17420 gleefully

18092 overwhelmed
18093 propagandize
18094 relocate
18095 spew
18096 statesmanship
18097 oboe
18098 kludgy
18099 severy
18100 window_manag
18101 frosty
18102 chorske
18103 agonize
18104 backhand
18105 backhander
18106 carryover
18107 clinch
18108 coudlnt
18109 crunching
18110 displeasure
18111 erry
18112 evetually
18113 faceoff
18114 hustle
18115 periods
18116 retaliatory
18117 shorthande
18118 shorthanded
18119 smother
18120 throug
18121 unflux
18122 wristed
18123 freon
18124 momentum
18125 stabilization
18126 telemetry
18127 bacground
18128 pantie
18129 quarterly
18130 backbone
18131 artform
18132 sizing
18133 mixing
18134 chillingly
18135 eugenics
18136 experiencce
18137 infantile
18138 judenrein
18139 outburst
18140 pragmatism
18141 uncivilized
18142 malmute
18143 steatopygiass
18144 defragmentation
18145 melanoma
18146 radonc
18147 reformatte
18148 utilite
18149 unset
18150 intergalactic
18151 anyone_tell
18152 netbio
18153 environet
18154 envnet
18155 telnette
18156 apex
18157 ch

In [33]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 4), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]]


In [34]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('bumper', 1),
  ('call', 1),
  ('car', 4),
  ('could', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('line', 1),
  ('look', 2),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('thank', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## Latent Dirichlet Allocation

![Latent Dirichlet Allocation](https://miro.medium.com/max/1400/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg)

In [35]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [36]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.014*"drug" + 0.012*"doctor" + 0.011*"blood" + 0.010*"water" + '
  '0.009*"soon" + 0.009*"effect" + 0.008*"patient" + 0.008*"candida" + '
  '0.008*"hot" + 0.008*"night"'),
 (1,
  '0.010*"may" + 0.010*"use" + 0.009*"write" + 0.008*"find" + 0.008*"case" + '
  '0.007*"many" + 0.007*"show" + 0.007*"must" + 0.007*"example" + '
  '0.007*"read"'),
 (2,
  '0.019*"file" + 0.018*"use" + 0.015*"problem" + 0.015*"window" + '
  '0.015*"line" + 0.015*"thank" + 0.014*"program" + 0.012*"work" + '
  '0.011*"card" + 0.010*"run"'),
 (3,
  '0.023*"american" + 0.021*"icon" + 0.018*"authority" + 0.017*"testing" + '
  '0.017*"animal" + 0.014*"modify" + 0.014*"extension" + 0.012*"ice" + '
  '0.011*"deserve" + 0.011*"door"'),
 (4,
  '0.016*"believe" + 0.016*"law" + 0.013*"word" + 0.011*"jewish" + '
  '0.011*"people" + 0.011*"argument" + 0.009*"faith" + 0.008*"truth" + '
  '0.008*"therefore" + 0.008*"follow"'),
 (5,
  '0.036*"car" + 0.034*"drive" + 0.021*"buy" + 0.020*"sell" + 0.014*"price" + '
  '0.01

In [46]:
lda_model.show_topic(9)

[('would', 0.019468188),
 ('write', 0.014832203),
 ('say', 0.011618762),
 ('get', 0.011593245),
 ('go', 0.01086993),
 ('think', 0.010151872),
 ('make', 0.009890938),
 ('be', 0.009890406),
 ('people', 0.00931426),
 ('well', 0.009108134)]

In [37]:
lda_model

In [38]:
doc_lda = lda_model[corpus]

In [39]:
#Perplejidad
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.730233194104606

Coherence Score:  0.46219614251068386


In [40]:
# Visualizamos los temas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis